In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
session.use_role('SYSADMIN')

In [ ]:
CREATE DATABASE IF NOT EXISTS MASTERCLASS
  DATA_RETENTION_TIME_IN_DAYS = 2
  COMMENT = 'Database for the Snowflake Masterclass';

CREATE WAREHOUSE IF NOT EXISTS MASTERCLASSWH
  WAREHOUSE_TYPE = STANDARD
  RESOURCE_CONSTRAINT = 'STANDARD_GEN_2'
  WAREHOUSE_SIZE = SMALL
  MAX_CLUSTER_COUNT = 2
  MIN_CLUSTER_COUNT = 1
  SCALING_POLICY = STANDARD
  AUTO_SUSPEND = 60
  AUTO_RESUME = TRUE
  INITIALLY_SUSPENDED = TRUE
  ENABLE_QUERY_ACCELERATION = TRUE
  QUERY_ACCELERATION_MAX_SCALE_FACTOR = 2
  COMMENT = 'Masterclass Warehouse';

CREATE FILE FORMAT IF NOT EXISTS MASTERCLASS.PUBLIC.GENERIC_CSV
 TYPE = 'CSV'
 COMPRESSION = AUTO
 RECORD_DELIMITER = '\n'
 FIELD_DELIMITER = ','
 MULTI_LINE = TRUE
 PARSE_HEADER = FALSE
 SKIP_HEADER = 1
 SKIP_BLANK_LINES = TRUE
 TRIM_SPACE = TRUE
 FIELD_OPTIONALLY_ENCLOSED_BY = '"'
 EMPTY_FIELD_AS_NULL = TRUE;

CREATE OR REPLACE STAGE MASTERCLASS.PUBLIC.INTERNAL_DATA_STAGE
  ENCRYPTION = (TYPE = 'SNOWFLAKE_SSE' )
  DIRECTORY  = (ENABLE = TRUE AUTO_REFRESH = TRUE)
  COMMENT = 'Stage to load data';

# LOAD FILE TO STAGE

# PROCESS FILE

In [ ]:
USE ROLE ACCOUNTADMIN;
USE WAREHOUSE MASTERCLASSWH;
USE SCHEMA MASTERCLASS.PUBLIC;

CREATE OR REPLACE TABLE MASTERCLASS.PUBLIC.CIRCUITS AS
SELECT
 NULLIF(t.$1::VARCHAR, '') AS CIRCUITID
,NULLIF(t.$2::VARCHAR, '') AS CIRCUITREF
,NULLIF(t.$3::VARCHAR, '') AS NAME
,NULLIF(t.$4::VARCHAR, '') AS LOCATION
,NULLIF(t.$5::VARCHAR, '') AS COUNTRY
,NULLIF(t.$6::VARCHAR, '') AS LAT
,NULLIF(t.$7::VARCHAR, '') AS LNG
,NULLIF(t.$8::VARCHAR, '') AS ALT
,NULLIF(t.$9::VARCHAR, '') AS URL
FROM '@"MASTERCLASS"."PUBLIC"."INTERNAL_DATA_STAGE"/circuits.csv' t;

In [ ]:
df = session.read.table("MASTERCLASS.PUBLIC.CIRCUITS")
df.limit(5)